##Packages

In [1]:
import random
import numpy as np
import torch
import gym
from gym import spaces
import torch.nn as nn
import torch.optim as optim
from collections import deque
import matplotlib.pyplot as plt
import pfhedge
from pfhedge.instruments import HestonStock, BrownianStock
from pfhedge.instruments import EuropeanOption
from pfhedge.nn import BlackScholes
from pfhedge.nn import Hedger
from pfhedge.nn import WhalleyWilmott
from collections import Counter


## Environment

In [2]:
class CustomHestonEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, k, kappa, sigma, theta):
        super(CustomHestonEnv, self).__init__()
        self.k = k
        self.action_space = spaces.Discrete(11)
        self.observation_space = spaces.Box(low=np.array([0, 0, 0]),
                                            high=np.array([np.inf, np.inf, np.inf]),
                                            dtype=np.float32)
        self.kappa = kappa
        self.sigma = sigma
        self.theta = theta
        self.reset()

    def pricer(self, derivative):
        return BlackScholes(derivative).price(
            log_moneyness=derivative.log_moneyness(),
            time_to_maturity=derivative.time_to_maturity(),
            volatility=derivative.ul().volatility)

    def step(self, action):
        if self.current_step == 0:
            self.H = action * 0.1
            reward = -self.k * abs(self.S_sim[0] * self.H)
            self.current_step += 1
            done = False
            next_state = np.array([action * 0.1, self.S_sim[1], 20 - self.current_step])
            return next_state, reward, done, {}
        elif self.current_step <= 18:
            S_prev = self.S_sim[self.current_step-1]
            V_prev = self.V_sim[self.current_step-1]
            S_current = self.S_sim[self.current_step]
            V_current = self.V_sim[self.current_step]
            H_current = action * 0.1
            H_prev = self.H

            reward = V_current - V_prev + H_prev * (S_current - S_prev) - self.k * abs(S_current * (H_current - H_prev))

            self.H = H_current
            self.current_step += 1
            done = False

            next_state = np.array([action * 0.1, self.S_sim[self.current_step], 20 - self.current_step])
            return next_state, reward, done, {}
        else:
            done = True
            reward = -1*self.k * abs(self.S_sim[-1] * self.H)
            return np.array([0,self.S_sim[-1],0]), reward, done, {}

    def reset(self):
        self.stock = HestonStock(cost=0.01, kappa=self.kappa, sigma=self.sigma, theta=self.theta)
        self.derivative = EuropeanOption(self.stock, maturity=20/250, call=True)
        self.derivative.list(self.pricer, cost=0)
        self.derivative.simulate(n_paths=1)

        self.S_sim = self.derivative.underlier.spot.detach().numpy().flatten()
        self.V_sim = -1*self.derivative.spot.detach().numpy().flatten()

        self.current_step = 0
        self.state = np.array([0, self.S_sim[0], 20])

        return self.state

    def render(self, mode='human'):
        pass

    def close(self):
        pass

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Learning

In [3]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        sample = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*sample)
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

    def __len__(self):
        return len(self.buffer)

class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


class QNetworkVariance(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetworkVariance, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.relu(self.fc3(x))

class DQNAgent:
    def __init__(self, state_dim, action_dim, replay_buffer):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.replay_buffer = replay_buffer
        self.q1_network = QNetwork(state_dim, action_dim)
        self.q2_network = QNetworkVariance(state_dim, action_dim)
        self.optimizer_q1 = optim.Adam(self.q1_network.parameters(), lr=0.00001)
        self.optimizer_q2 = optim.Adam(self.q2_network.parameters(), lr=0.00001)
        self.gamma = 0
        self.criterion = nn.MSELoss()
        self.c = 1.5
        self.initialize_weights()

    def initialize_weights(self):
        def init_fn(m):
            if isinstance(m, nn.Linear):
                torch.nn.init.uniform_(m.weight, a=0.01, b=0.01)
                torch.nn.init.constant_(m.bias, 0.01)

        self.q1_network.apply(init_fn)
        self.q2_network.apply(init_fn)

    def learn(self, batch_size):
        if len(self.replay_buffer) < batch_size:
            return
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(batch_size)

        states = torch.FloatTensor(states)
        next_states = torch.FloatTensor(next_states)
        actions = torch.LongTensor(actions)
        rewards = -1*torch.FloatTensor(rewards)
        dones = torch.FloatTensor(dones)

        current_q1_values = self.q1_network(states)
        current_q2_values = self.q2_network(states)

        current_q1_values = current_q1_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        current_q2_values = current_q2_values.gather(1, actions.unsqueeze(1)).squeeze(1)

        with torch.no_grad():
          q1_values = self.q1_network(states)
          q2_values = self.q2_network(states)
          f_values = q1_values + self.c * torch.sqrt(torch.clamp((q2_values), min=0))
          next_actions = f_values.argmin(1, keepdim=True)


        next_q1_values = self.q1_network(next_states).gather(1, next_actions).squeeze(1)
        next_q2_values = self.q2_network(next_states).gather(1, next_actions).squeeze(1)

        expected_q1_values = rewards + self.gamma * next_q1_values
        expected_q2_values = rewards.pow(2) + next_q2_values + 2 * rewards * next_q1_values

        loss1 = self.criterion(current_q1_values, expected_q1_values.detach())
        loss2 = self.criterion(current_q2_values, expected_q2_values.detach())

        self.optimizer_q1.zero_grad()
        loss1.backward()
        self.optimizer_q1.step()

        self.optimizer_q2.zero_grad()
        loss2.backward()
        self.optimizer_q2.step()

        return loss1.item(), loss2.item()

    def select_action(self, state, epsilon, env):
        if random.random() > epsilon:
            state = torch.FloatTensor(state).unsqueeze(0)
            with torch.no_grad():
                q1_values = self.q1_network(state)
                q2_values = self.q2_network(state)
                f_values = q1_values + 1.5 * torch.sqrt(torch.clamp((q2_values), min=0))
                action = f_values.argmin(dim=1).item()
        else:
            action = env.action_space.sample()
        return action

## Results


In [4]:
def compute_delta_hedging_objective_function(env, c, n):
    costs = []

    for j in range(n):
      state = env.reset()
      model = BlackScholes(env.derivative)
      hedger = Hedger(model, inputs=model.inputs())
      actions = hedger.compute_hedge(env.derivative).detach().numpy().flatten()*10
      done = False
      total_hedging_cost = 0
      i=0

      while not done:
          action = actions[i]
          state, reward, done, _ = env.step(action)
          total_hedging_cost -= reward
          i+=1

      costs.append(total_hedging_cost)

    expected_cost = np.mean(costs)
    std_dev_cost = np.std(costs)

    Y_0 = expected_cost + c * std_dev_cost
    return (Y_0, expected_cost, std_dev_cost)


def compute_objective_function_Q_learning(env, agent, c, n):
    costs = []
    actions = []

    for j in range(n):
      state = env.reset()
      done = False
      total_hedging_cost = 0

      while not done:
          action = agent.select_action(state, -1, env)
          actions.append(action)
          state, reward, done, _ = env.step(action)
          total_hedging_cost -= reward

      costs.append(total_hedging_cost)

    expected_cost = np.mean(costs)
    std_dev_cost = np.std(costs)

    print(Counter(actions))

    Y_0 = expected_cost + c * std_dev_cost
    return (Y_0, expected_cost, std_dev_cost)

def train_agent(kappa, sigma, theta):
    env = CustomHestonEnv(k=0.01, kappa=kappa, sigma=sigma, theta=theta)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    buffer_size = 256
    replay_buffer = ReplayBuffer(buffer_size)
    agent = DQNAgent(state_dim, action_dim, replay_buffer)

    episodes = 1500
    epsilon = 1
    epsilon_decay = 0.995
    epsilon_min = 0.01
    batch_size = 32
    total_losses = []


    while len(replay_buffer) < batch_size:
        state = env.reset()
        for _ in range(100):
            action = env.action_space.sample()
            next_state, reward, done, _ = env.step(action)
            replay_buffer.push(state, action, reward, next_state, done)
            state = next_state
            if done:
                break

    for episode in range(episodes):
        state = env.reset()
        total_loss1, total_loss2 = 0, 0
        steps = 0
        total_reward = 0

        while True:
            if episode > 100:
              agent.gamma = 1
            action = agent.select_action(state, epsilon, env)
            next_state, reward, done, _ = env.step(action)
            replay_buffer.push(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            loss1, loss2 = agent.learn(batch_size)
            total_loss1 += loss1
            total_loss2 += loss2
            steps += 1

            if done:
                break
        total_losses.append(total_loss2)

        epsilon = max(epsilon_min, epsilon * epsilon_decay)

        if (episode + 1) % 50 == 0:
          env.reset()
          objective_value_q_learning = compute_objective_function_Q_learning(env, agent, 1.5, 500)
          objective_value_delta = compute_delta_hedging_objective_function(env, 1.5, 100)
          print(f'Objective function value at episode {episode + 1}: {objective_value_q_learning} versus {objective_value_delta} and mean loss: {sum(total_losses[episode-100:episode])/100}')

          if objective_value_q_learning[0] < objective_value_delta[0]:
            return agent

    env.close()

    return agent


In [ ]:
results = []
theta = 0.08
for kappa in [1, 1.5, 2]:
  for sigma in [0.2, 0.3, 0.4]:
    agent = train_agent(kappa=kappa, sigma=sigma, theta=theta)
    env = CustomHestonEnv(k=0.01, kappa=kappa, sigma=sigma, theta=theta)
    results.append({"kappa": kappa,
                    "sigma": sigma,
                    "theta": theta,
                    "delta-hedging":compute_delta_hedging_objective_function(env, 1.5, 1000),
                    "q-learning":compute_objective_function_Q_learning(env, agent, 1.5, 1000)
                    })
    print(results[-1])


Counter({1: 10000})
Objective function value at episode 50: (0.056707695671034765, 0.002576676163990931, 0.03608734633802922) versus (0.034347045799361015, 0.022029372043690095, 0.00821178250378061) and mean loss: 3.5397717731910914e-06
Counter({3: 10000})
Objective function value at episode 100: (0.040161079696408336, 0.003457349219155723, 0.024469153651501742) versus (0.03767196409707832, 0.024648149138525283, 0.008682543305702025) and mean loss: 0.0
Counter({6: 8500, 5: 1500})
Objective function value at episode 150: (0.04276227087308436, 0.01051312216919433, 0.02149943246926002) versus (0.038915353633903124, 0.02460921529183839, 0.00953742556137649) and mean loss: 1.0664592374665105e-06
Counter({4: 10000})
Objective function value at episode 200: (0.04048437797812615, 0.00790211256795016, 0.021721510273450657) versus (0.03228639457799817, 0.02256467505784322, 0.00648114634676997) and mean loss: 1.568835203555885e-06
Counter({6: 4500, 4: 4500, 3: 1000})
Objective function value at e

In [ ]:
cleaned_results = []
for result in results:
  cleaned_results.append({'kappa': result['kappa'], 'sigma': result['sigma'], 'theta': result['theta'],
                          'delta-hedging-objective-value': result['delta-hedging'][0], 'q-learning-objective-value': result['q-learning'][0],
                          'delta-hedging-mean-cost': result['delta-hedging'][1], 'q-learning-mean-cost': result['q-learning'][1],
                          'delta-hedging-std-dev-cost': result['delta-hedging'][2], 'q-learning-std-dev-cost': result['q-learning'][2]
                          })
import pandas as pd
pd.DataFrame(cleaned_results)